In [1]:
from random import seed
from random import random
from math import exp
import numpy as np
import pandas as pd
import textwrap
import timeit
import binascii
import json

In [2]:
# -*- coding: utf-8 -*-
import json

# Make it work for Python 2+3 and with Unicode
import io
try:
    to_unicode = unicode
except NameError:
    to_unicode = str

In [3]:
def addZeros(strr, n): 
    for i in range(n): 
        strr = "0" + strr 
    return strr 

def getXOR(a, b): 
    aLen = len(a) 
    bLen = len(b) 


    if (aLen > bLen): 
        b = addZeros(b, aLen - bLen) 
    elif (bLen > aLen): 
        a = addZeros(a, bLen - aLen) 
 
    lenn = max(aLen, bLen); 

    res = "" 
    for i in range(lenn): 
        if (a[i] == b[i]): 
            res += "0"
        else: 
            res += "1"
    return res 

In [4]:
def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

In [5]:
#initializing the network
def initialize_network(parameters):
    network = list()
    for j in range(1,len(parameters)):
        layer = [{'weights':[random() for i in range(parameters[j-1] + 1)]} for i in range(parameters[j])]
        network.append(layer)
    return network

In [6]:
# Make a prediction with a network
def predict(network, row, encr):
    outputs = forward_propagate(network, row, encr)
    k = [round(i) for i in outputs]
    return k

In [7]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [8]:
# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [9]:
# Forward propagate input to a network output
def forward_propagate(network, row, getEncryption):
    count = 0
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        if count == 1 and getEncryption == 1:
            print("io")
            return new_inputs
        inputs = new_inputs
        count += 1
  
    return inputs

In [10]:
# Update network weights with error
def update_weights(network, row, l_rate):
    
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    count = 1
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            count += 1
            outputs = forward_propagate(network, row, 0)
            expected = row[-1]
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        if count % 100 == 0:
            print('> epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
        count += 1

In [11]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [12]:
# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [13]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    #print(inputs)
    activation = weights[-1]
    for i in range(len(weights)-1):
        #print(i)
        #print(type(weights[i]))
        #print(type(inputs[i]))
        activation += weights[i] * inputs[i]
    return activation

In [14]:
#batch training
# Train a network for a fixed number of epochs
def train_network_batch(network, train, l_rate, n_epoch, n_outputs):
    count = 1
    k = 0
    batch = 2
    for i in range(batch):
        print('> Batch = %d' % (i+1))
        for epoch in range(n_epoch):
            sum_error = 0
            for row in train[k:k+10]:
                count += 1
                outputs = forward_propagate(network, row, 0)
                expected = row[-1]
                sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
                backward_propagate_error(network, expected)
                update_weights(network, row, l_rate)
            if count % 100 == 0:
                print('    epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
            count += 1
        k += 10

In [87]:
# Read JSON file
with open('../network/network.json') as data_file:
    network = json.load(data_file)

In [106]:
# Read JSON file
with open('../datasets/languageDecryptor.json') as data_file:
    dataset = json.load(data_file)

with open('../datasets/languageDecryptor.json') as data_file:
    temp = json.load(data_file)

In [76]:
for i in range(100):
    k = temp[i][-1]
    k.append(temp[i][:-1])
    dataset.append(k)

In [89]:
len(dataset)

100

In [123]:
seed(1)
network = initialize_network([8,18,18,8])

In [124]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(dataset[0][-1])
start = timeit.default_timer()
train_network(network, dataset, 0.5, 10000, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

> epoch=99, lrate=0.500, error=223.557
> epoch=199, lrate=0.500, error=174.314
> epoch=299, lrate=0.500, error=158.024
> epoch=399, lrate=0.500, error=92.825
> epoch=499, lrate=0.500, error=46.072
> epoch=599, lrate=0.500, error=17.516
> epoch=699, lrate=0.500, error=13.383
> epoch=799, lrate=0.500, error=11.074
> epoch=899, lrate=0.500, error=9.811
> epoch=999, lrate=0.500, error=9.603
> epoch=1099, lrate=0.500, error=9.480
> epoch=1199, lrate=0.500, error=9.396
> epoch=1299, lrate=0.500, error=9.282
> epoch=1399, lrate=0.500, error=8.300
> epoch=1499, lrate=0.500, error=7.283
> epoch=1599, lrate=0.500, error=7.227
> epoch=1699, lrate=0.500, error=7.198
> epoch=1799, lrate=0.500, error=7.178
> epoch=1899, lrate=0.500, error=7.162
> epoch=1999, lrate=0.500, error=7.149
> epoch=2099, lrate=0.500, error=7.139
> epoch=2199, lrate=0.500, error=7.130
> epoch=2299, lrate=0.500, error=7.122
> epoch=2399, lrate=0.500, error=7.115
> epoch=2499, lrate=0.500, error=7.109
> epoch=2599, lrate=0.500

In [130]:
c = 0
for i in dataset:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        c += 1
print(str(c)+"% error")

6% error


In [136]:
c = 0
new_dataset = list()
for i in dataset:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        new_dataset.append(i)
        print("index = %d" % (c))
        print(k)
        print(j)
        print()
    c += 1

index = 4
00110110
00110100

index = 11
01110010
01100010

index = 13
01100110
01100100

index = 34
01111011
01111001

index = 35
01110010
01111010

index = 64
00100111
00100011



In [55]:
with open('../datasets/languageDecryptor.json') as data_file:
    temp1 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp2 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp3 = json.load(data_file)
    
with open('../datasets/languageDecryptor.json') as data_file:
    temp4 = json.load(data_file)

In [17]:
temp1

[[1, 1, 0, 1, 1, 1, 0, 1, [0, 0, 1, 1, 0, 0, 0, 0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 0, 1]],
 [1, 1, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 0]],
 [1, 0, 1, 1, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 0]],
 [1, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0, 0, 1, 1, 1, 0, 0, 0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0, 0, 1, 1, 1, 0, 0, 1]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0, 1, 1, 0, 0, 0, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 1, [0, 1, 1, 0, 0, 0, 1, 0]],
 [1, 0, 0, 0, 1, 1, 0, 1, [0, 1, 1, 0, 0, 0, 1, 1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0, 1, 1, 0, 0, 1, 0, 0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0, 1, 1, 0, 0, 1, 0, 1]],
 [1, 0, 0, 0, 0, 0, 1, 1, [0, 1, 1, 0, 0, 1, 1, 0]],
 [1, 0, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 0, 1, 1, 1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 1, 0, 0, 0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0, 1, 1, 0, 1, 0, 0

In [56]:
for i in range(100):
    temp1[i][-1] = temp1[i][-1][:2]
    temp2[i][-1] = temp2[i][-1][2:4]
    temp3[i][-1] = temp3[i][-1][4:6]
    temp4[i][-1] = temp4[i][-1][6:]

In [57]:
temp3

[[1, 1, 0, 1, 1, 1, 0, 1, [0, 0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0, 0]],
 [1, 1, 0, 0, 0, 1, 1, 0, [0, 0]],
 [1, 0, 1, 1, 1, 1, 1, 0, [0, 0]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0, 1]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 0, [0, 1]],
 [1, 0, 0, 0, 0, 1, 1, 0, [0, 1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [1, 0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [1, 0]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0, 0]],
 [1, 0, 0, 1, 0, 1, 1, 1, [0, 0]],
 [1, 0, 0, 0, 1, 1, 0, 1, [0, 0]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0, 1]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0, 1]],
 [1, 0, 0, 0, 0, 0, 1, 1, [0, 1]],
 [1, 0, 0, 1, 1, 1, 0, 0, [0, 1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [1, 0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [1, 0]],
 [1, 0, 0, 0, 0, 0, 0, 0, [1, 0]],
 [1, 1, 0, 1, 0, 0, 1, 0, [1, 0]],
 [1, 1, 1, 1, 0, 1, 0, 1, [1, 1]],
 [1, 1, 0, 1, 0, 1, 0, 1, [1, 1]],
 [0, 0, 0, 1, 1, 0, 0, 0, [1, 1]],
 [1, 1, 0, 1, 0, 0, 0, 1, [1, 1]],
 [1, 0, 1, 1, 1, 1, 1, 1, [0, 0]],
 [1, 1, 1, 1, 0, 0, 0, 1, [0, 0]],
 [1, 0, 0, 1, 0, 1, 0, 1, [0, 0]],
 [0, 0, 0, 1, 0, 1, 

In [151]:
# Read JSON file
with open('../network/network.json') as data_file:
    network2 = json.load(data_file)

In [309]:
seed(1)
network = initialize_network([8,9,3,2])

In [310]:

n_inputs = len(temp[0]) - 1
n_outputs = len(temp[0][-1])
start = timeit.default_timer()
train_network(network, temp1, 0.5, 10000, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

> epoch=99, lrate=0.500, error=24.672
> epoch=199, lrate=0.500, error=19.269
> epoch=299, lrate=0.500, error=18.045
> epoch=399, lrate=0.500, error=15.061
> epoch=499, lrate=0.500, error=5.394
> epoch=599, lrate=0.500, error=4.931
> epoch=699, lrate=0.500, error=4.878
> epoch=799, lrate=0.500, error=4.793
> epoch=899, lrate=0.500, error=4.235
> epoch=999, lrate=0.500, error=3.015
> epoch=1099, lrate=0.500, error=2.872
> epoch=1199, lrate=0.500, error=2.055
> epoch=1299, lrate=0.500, error=1.983
> epoch=1399, lrate=0.500, error=1.936
> epoch=1499, lrate=0.500, error=1.049
> epoch=1599, lrate=0.500, error=1.019
> epoch=1699, lrate=0.500, error=0.978
> epoch=1799, lrate=0.500, error=0.848
> epoch=1899, lrate=0.500, error=0.045
> epoch=1999, lrate=0.500, error=0.026
> epoch=2099, lrate=0.500, error=0.019
> epoch=2199, lrate=0.500, error=0.015
> epoch=2299, lrate=0.500, error=0.013
> epoch=2399, lrate=0.500, error=0.011
> epoch=2499, lrate=0.500, error=0.010
> epoch=2599, lrate=0.500, error

In [312]:
c = 0
new_dataset = list()
for i in temp1:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        print(k)
        print(j)
        new_dataset.append(i)
        c += 1
print(str(c)+" %")

0 %


In [315]:
# Write JSON file
with io.open('../network/decryptor/network1.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(network,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))

In [58]:
seed(1)
network = initialize_network([8,9,7,2])

In [ ]:

n_inputs = len(temp2[0]) - 1
n_outputs = len(temp2[0][-1])
start = timeit.default_timer()
train_network(network, temp3, 1, 30000, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

> epoch=99, lrate=1.000, error=47.782
> epoch=199, lrate=1.000, error=29.489
> epoch=299, lrate=1.000, error=17.290
> epoch=399, lrate=1.000, error=11.760
> epoch=499, lrate=1.000, error=7.653
> epoch=599, lrate=1.000, error=7.402
> epoch=699, lrate=1.000, error=6.179
> epoch=799, lrate=1.000, error=5.963
> epoch=899, lrate=1.000, error=5.884
> epoch=999, lrate=1.000, error=5.847
> epoch=1099, lrate=1.000, error=5.824
> epoch=1199, lrate=1.000, error=5.808
> epoch=1299, lrate=1.000, error=5.797
> epoch=1399, lrate=1.000, error=5.788
> epoch=1499, lrate=1.000, error=5.781
> epoch=1599, lrate=1.000, error=5.776
> epoch=1699, lrate=1.000, error=5.772
> epoch=1799, lrate=1.000, error=5.768
> epoch=1899, lrate=1.000, error=5.765
> epoch=1999, lrate=1.000, error=5.762
> epoch=2099, lrate=1.000, error=5.760
> epoch=2199, lrate=1.000, error=5.758
> epoch=2299, lrate=1.000, error=5.756
> epoch=2399, lrate=1.000, error=5.755
> epoch=2499, lrate=1.000, error=5.753
> epoch=2599, lrate=1.000, error

In [39]:
c = 0
new_dataset = list()
for i in temp2:
    k = predict(network,i,0)
    s = ""
    k = list(map(str,k))
    k = s.join(k)
    j = i[-1]
    s = ""
    j= list(map(str,j))
    j = s.join(j)
    if(k != j):
        print(k)
        print(j)
        new_dataset.append(i)
        c += 1
print(str(c)+" %")

0 %


In [45]:
# Write JSON file
with io.open('../network/decryptor/network2.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(network,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))